# Install and load necesary packages

In [1]:
# Please don't change this cell

import pandas as pd
import numpy as np  

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Please don't change this cell
df = pd.read_csv('ml-100k/u.data', names=['user_id', 'item_id', 'rating', 'timestamp'], sep='\t')

df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


# Split dataset
## Random Train and Test Split

In [3]:
# please do not change this cell

from sklearn.model_selection import train_test_split

n_users = df.user_id.unique().shape[0]
n_items = df.item_id.unique().shape[0]
print(str(n_users) + ' users')
print(str(n_items) + ' items')

train_df, test_df = train_test_split(df, test_size=0.2, random_state = 10)
train_df, test_df

# Training Dataset
train_ds = np.zeros((n_users, n_items))
item_popularity = np.zeros(n_items)
for row in train_df.itertuples():
    train_ds[row[1]-1, row[2]-1] = row[3]
    item_popularity[row[2]-1] =  item_popularity[row[2]-1] + 1
#train_ds = pd.DataFrame(train_ds)

# Testing Dataset
testsize = 0
test_ds = np.zeros((n_users, n_items))
for row in test_df.itertuples():
    if item_popularity[row[2]-1] > 30:
        test_ds[row[1]-1, row[2]-1] = row[3]
        testsize = testsize + 1
#test_ds = pd.DataFrame(test_ds)

print("Construct the rating matrix based on train_df:")
print(train_ds)

print("Construct the rating matrix based on test_df:")
print(test_ds)

print("Testsize = " + str(testsize))

943 users
1682 items
Construct the rating matrix based on train_df:
[[0. 3. 4. ... 0. 0. 0.]
 [4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 5. 0. ... 0. 0. 0.]]
Construct the rating matrix based on test_df:
[[5. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Testsize = 17678


# Utils

In [4]:
# Please don't change this cell
# you can use this devaluate Utils here, and you can also implement your own MAE and RMSE calculation. 

EPSILON = 1e-9

def evaluate(test_ds, predicted_ds):
    '''
    Function for evaluating on MAE and RMSE
    '''
    # MAE
    mask_test_ds = test_ds > 0
    MAE = np.sum(np.abs(test_ds[mask_test_ds] - predicted_ds[mask_test_ds])) / np.sum(mask_test_ds.astype(np.float32))

    # RMSE
    RMSE = np.sqrt(np.sum(np.square(test_ds[mask_test_ds] - predicted_ds[mask_test_ds])) / np.sum(mask_test_ds.astype(np.float32)))

    return MAE, RMSE

# Your Solution

In [5]:
# Write your code here
# You are required to implement the required solution here. 
# Then, evaluate your implementation by predicting the ratings in the test set (test_ds).
# Finally, save the corresponding MAE and RMSE of your implementation 
# into the following defined corresponding variable. 

MAE = 0 # 0 is an intial value, you need to update this with the actual performance of your implementation.
RMSE = 0 # 0 is an intial value, you need to update this with the actual performance of your implementation.

#converting to a data frame
train_ds = pd.DataFrame(train_ds)
test_ds = pd.DataFrame(test_ds)

In [6]:
#importing necessary libraries
import numpy as np
import pandas as pd

# Constants
EPSILON = 1e-9
Vmin = 1  # Assuming ratings are on a scale of 1 to 5
Vmax = 5

# Assuming train_ds and test_ds are provided as Pandas DataFrames
n_users, n_items = train_ds.shape

# Calculate the similarity matrix using the formula in the report

#initialising the user-user similarity matrix with zero 
np_user_aed_corr = np.zeros((n_users, n_users))
#initialising the Euclidean_distances matrix to store the Euclidean distance between every pair of user
euclidean_distances = np.zeros((n_users, n_users))

#iterating through the list to find the similarity between user-user
for i, user_i_vec in enumerate(train_ds.values):
    for j, user_j_vec in enumerate(train_ds.values):

        #when comparing the same user set the similarity to 1
        if i == j:
            np_user_aed_corr[i][j] = 1
            euclidean_distances[i][j] = 0
            continue
        # Mask for common rated items
        mask_i = user_i_vec > 0
        mask_j = user_j_vec > 0
        common_mask = mask_i & mask_j
        #if the users have no ratings in common then let the similarity remain 0
        if np.sum(common_mask) == 0:
            continue
        # Calculate the numerator of the AED formula
        sum_diff_squared = np.sum((user_i_vec[common_mask] - user_j_vec[common_mask]) ** 2)
        # Calculate the denominator of the AED formula
        dist_max = np.sqrt(np.sum(common_mask)) * (Vmax - Vmin)
        aed = np.sqrt(sum_diff_squared) / (dist_max + EPSILON)
        #find the similarity score based on the aed
        sim = 1 - aed
        np_user_aed_corr[i][j] = sim
        euclidean_distances[i][j] = np.sqrt(sum_diff_squared)

# Output the similarity matrix
print(np_user_aed_corr)


[[1.         0.7236146  0.44098301 ... 0.72613872 0.5669873  0.60543465]
 [0.7236146  1.         0.5669873  ... 0.51587708 0.627322   0.85566243]
 [0.44098301 0.5669873  1.         ... 0.60471529 0.50900975 1.        ]
 ...
 [0.72613872 0.51587708 0.60471529 ... 1.         0.75       0.68377223]
 [0.5669873  0.627322   0.50900975 ... 0.75       1.         0.7058258 ]
 [0.60543465 0.85566243 1.         ... 0.68377223 0.7058258  1.        ]]


In [7]:
# Predict ratings
np_predictions = np.zeros((n_users, n_items))
#Considering 30 similar neighbours
K = 30

for (i, j), rating in np.ndenumerate(test_ds.values):
    if rating > 0:  # Predict only for rated items
        # Find top-K most similar users
        sim_user_ids = np.argsort(np_user_aed_corr[i])[-(K + 1):-1]
        #similarity values wrt i
        sim_val = np_user_aed_corr[i][sim_user_ids]
        #rating vectors of the similar user
        sim_users = train_ds.values[sim_user_ids]

        #mean rating of user i
        user_mean = np.sum(train_ds.values[i]) / (np.sum(np.clip(train_ds.values[i], 0, 1)) + EPSILON)
        #subset of similar users who have rated item j
        mask_rated_j = sim_users[:, j] > 0

        # Check for users with similarity == 1(or euclidean distance ==0) and also rated the movie j
        identical_users_mask = (np_user_aed_corr[i][sim_user_ids] == 1) & mask_rated_j

        #for identical users
        if np.any(identical_users_mask):

            #rating vectors of the identical users
            identical_users = sim_users[identical_users_mask]
            #subset of identical users who have rated item j
            identical_users_ratings = identical_users[:, j] 

            #number of co-rated items between user i and identical user
            m_c_values = np.array([np.sum((train_ds.values[i] > 0) & (train_ds.values[user_id] > 0)) for user_id in sim_user_ids[identical_users_mask]])
            #if co-rated items exist Using the formula for identical users from the report
            if np.sum(m_c_values) > 0:
                k0 = 1 / np.sum(m_c_values)
                np_predictions[i][j] = k0 * np.sum(m_c_values * identical_users_ratings)
            else:
                # Use the weighted sum formula(considering similar users) when identical users have not rated the item
                weighted_sum_ratings = sim_val[mask_rated_j] * sim_users[mask_rated_j, j]
                k = 1 / np.sum(sim_val[mask_rated_j])
                np_predictions[i][j] = k * np.sum(weighted_sum_ratings)
        else:
            # Standard weighted sum(considering similar users) calculation or when there are no identical users
            if np.sum(mask_rated_j) > 0:
                weighted_sum_ratings = sim_val[mask_rated_j] * sim_users[mask_rated_j, j]
                k = 1 / np.sum(sim_val[mask_rated_j])
                np_predictions[i][j] = k * np.sum(weighted_sum_ratings)
            #if none of the similar users have rated item j then assign the user_mean as the prediction
            else:
                np_predictions[i][j] = user_mean
        
        # Clip the prediction to the valid rating range
        np_predictions[i][j] = np.clip(np_predictions[i][j], 0, 5)

print(np_predictions)


[[4. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [8]:
labels = test_ds.values

# absolute error on all ratings
absolute_error = np.abs(np_predictions - labels)

# weight
weight = np.clip(labels, 0, 1)

# absoulte error on rated ratings
abs_error = absolute_error * weight

# MAE
MAE = np.sum(abs_error) / np.sum(weight)

print("MAE on Tesing set (User-based): when K=30 and random state=10 " + str(MAE));

MAE on Tesing set (User-based): when K=30 and random state=10 0.856048641065022


In [9]:
#==================RMSE on Testing set===================
labels = test_ds.values

# squared error on all ratings
squared_error = np.square(np_predictions - labels)
weight = np.clip(labels, 0, 1)

# squared error on rated ratings
squared_error = squared_error * weight

# RMSE
RMSE = np.sqrt(np.sum(squared_error) / np.sum(weight))

print("RMSE on Tesing set (User-based): when K=30 and random state=10 " + str(RMSE));

RMSE on Tesing set (User-based): when K=30 and random state=10 1.1185266906241733


In [10]:
# Please don't change this cell

print("===================== The MAE and RMSE of Your Implementation =====================")
print("MAE: {}, RMSE: {}" .format(MAE, RMSE))

===================== The MAE and RMSE of Your Implementation =====================
MAE: 0.856048641065022, RMSE: 1.1185266906241733
